In [45]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

import statsmodels.formula.api as smf
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.linear_model import LinearRegression, LogisticRegression

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, matthews_corrcoef


from matplotlib.lines import Line2D
from matplotlib.patches import Patch

In [51]:
phase2_df1=pd.read_csv('phase2_df(null).csv',low_memory=False)

In [47]:
# phase2_df1 = phase2_df1[
#     ~((phase2_df1['light_regime'] == '10min-10min') & (phase2_df1['y2_87'].notnull()))
# ]

In [48]:
from scipy import interpolate
from scipy.stats import rankdata

def normalize_quantiles(A, ties=True):
    A = np.asarray(A, dtype=np.float64)
    n_rows, n_cols = A.shape
    if n_cols == 1:
        return A.copy()

    i = np.linspace(0, 1, n_rows)
    S = np.full((n_rows, n_cols), np.nan)
    nobs = np.zeros(n_cols, dtype=int)
    sort_idx = []

    for j in range(n_cols):
        col = A[:, j]
        not_nan = ~np.isnan(col)
        x = col[not_nan]
        nobs[j] = len(x)
        sort_order = np.argsort(x)
        sorted_x = x[sort_order]

        if nobs[j] < n_rows:
            f = interpolate.interp1d(np.linspace(0, 1, nobs[j]), sorted_x,
                                     bounds_error=False, fill_value="extrapolate")
            S[:, j] = f(i)
        else:
            S[:, j] = sorted_x

        sort_idx.append(np.argsort(np.argsort(col[not_nan])))

    m = np.nanmean(S, axis=1)
    A_out = np.full_like(A, np.nan)

    for j in range(n_cols):
        col = A[:, j]
        not_nan = ~np.isnan(col)

        if ties:
            r = rankdata(col[not_nan], method='average')
            quant_pos = (r - 1) / (nobs[j] - 1)
            f = interpolate.interp1d(i, m, bounds_error=False, fill_value="extrapolate")
            A_out[not_nan, j] = f(quant_pos)
        else:
            ranks = sort_idx[j]
            A_out[not_nan, j] = m[ranks.astype(int)]

    return A_out

In [49]:
## drop 10min-10min
plates = ['31v1', '31v2', '31v3']
df_31v =phase2_df1[phase2_df1['plate'].isin(plates)]

group_counts = (
    df_31v.groupby(['plate', 'light_regime', 'mutant_ID', 'mutated_genes'])
    .size()
    .reset_index(name='count')
)

# Step 3: For each plate and light_regime, count how many mutants had 1, 2, ... rows
summary = (
    group_counts.groupby(['light_regime','plate', 'count'])
    .size()
    .reset_index(name='n_mutants')
)

# Optional: Sort for easier reading
summary = summary.sort_values(by=['light_regime','plate', 'count'])

# Show result
summary

,light_regime,plate,count,n_mutants
0,10min-10min,31v1,1,9
1,10min-10min,31v2,1,359
2,10min-10min,31v2,7,1
3,10min-10min,31v3,1,359
4,10min-10min,31v3,7,1
5,1min-1min,31v1,1,359
6,1min-1min,31v1,7,1
7,1min-1min,31v2,1,359
8,1min-1min,31v2,7,1
9,1min-1min,31v3,1,359


In [54]:

plates = ['31v1', '31v2', '31v3']
df_30v =phase2_df1[phase2_df1['plate'].isin(plates)]

group_counts = (
    df_30v.groupby(['plate', 'light_regime', 'mutant_ID', 'mutated_genes'])
    .size()
    .reset_index(name='count')
)

# Step 3: For each plate and light_regime, count how many mutants had 1, 2, ... rows
summary = (
    group_counts.groupby(['light_regime','plate', 'count'])
    .size()
    .reset_index(name='n_mutants')
)

# Optional: Sort for easier reading
summary = summary.sort_values(by=['light_regime','plate', 'count'])

# Show result
summary

,light_regime,plate,count,n_mutants
0,10min-10min,31v1,1,359
1,10min-10min,31v1,7,1
2,10min-10min,31v2,1,359
3,10min-10min,31v2,7,1
4,10min-10min,31v3,1,359
5,10min-10min,31v3,7,1
6,1min-1min,31v1,1,359
7,1min-1min,31v1,7,1
8,1min-1min,31v2,1,359
9,1min-1min,31v2,7,1


In [11]:
phase2_df1[(phase2_df1['light_regime']=='5min-5min')&(phase2_df1['plate']=='31v2')&(phase2_df1['mutant_ID']=='WT')]

,plate,measurement,start_date,light_regime,dark_threshold,light_threshold,num_frames,i,j,fv_fm,...,measurement_time_173,measurement_time_174,measurement_time_175,measurement_time_176,measurement_time_177,well_id,mutant_ID,feature,mutated_genes,num_mutations
18433,31v2,M8,2024-08-19,5min-5min,16.224237,22.57217,180,2,2,0.679190,...,NaN,NaN,NaN,NaN,NaN,C03,WT,NaN,"Cre09.g392000,Cre09.g406500,Cre02.g106400",3.0
18442,31v2,M8,2024-08-19,5min-5min,16.224237,22.57217,180,2,11,0.678890,...,NaN,NaN,NaN,NaN,NaN,C12,WT,NaN,"Cre09.g392000,Cre09.g406500,Cre02.g106400",3.0
18452,31v2,M8,2024-08-19,5min-5min,16.224237,22.57217,180,2,21,0.684181,...,NaN,NaN,NaN,NaN,NaN,C22,WT,NaN,"Cre09.g392000,Cre09.g406500,Cre02.g106400",3.0
18562,31v2,M8,2024-08-19,5min-5min,16.224237,22.57217,180,7,11,0.663624,...,NaN,NaN,NaN,NaN,NaN,H12,WT,NaN,"Cre09.g392000,Cre09.g406500,Cre02.g106400",3.0
18697,31v2,M8,2024-08-19,5min-5min,16.224237,22.57217,180,13,2,0.669593,...,NaN,NaN,NaN,NaN,NaN,N03,WT,NaN,"Cre09.g392000,Cre09.g406500,Cre02.g106400",3.0
18706,31v2,M8,2024-08-19,5min-5min,16.224237,22.57217,180,13,11,0.655085,...,NaN,NaN,NaN,NaN,NaN,N12,WT,NaN,"Cre09.g392000,Cre09.g406500,Cre02.g106400",3.0
18716,31v2,M8,2024-08-19,5min-5min,16.224237,22.57217,180,13,21,0.652171,...,NaN,NaN,NaN,NaN,NaN,N22,WT,NaN,"Cre09.g392000,Cre09.g406500,Cre02.g106400",3.0


In [12]:
phase2_df1[(phase2_df1['light_regime']=='5min-5min')&(phase2_df1['plate']=='31v3')&(phase2_df1['mutant_ID']=='WT')]

,plate,measurement,start_date,light_regime,dark_threshold,light_threshold,num_frames,i,j,fv_fm,...,measurement_time_173,measurement_time_174,measurement_time_175,measurement_time_176,measurement_time_177,well_id,mutant_ID,feature,mutated_genes,num_mutations
21880,31v3,M8,2024-09-04,5min-5min,15.602963,21.738926,180,2,2,0.659853,...,NaN,NaN,NaN,NaN,NaN,C03,WT,MULTIPLE_SPLICE_VARIANTS,"Cre09.g392000,Cre09.g406500,Cre02.g106400",3.0
21889,31v3,M8,2024-09-04,5min-5min,15.602963,21.738926,180,2,11,0.672874,...,NaN,NaN,NaN,NaN,NaN,C12,WT,intron,"Cre09.g392000,Cre09.g406500,Cre02.g106400",3.0
21899,31v3,M8,2024-09-04,5min-5min,15.602963,21.738926,180,2,21,0.655524,...,NaN,NaN,NaN,NaN,NaN,C22,WT,MULTIPLE_SPLICE_VARIANTS,"Cre09.g392000,Cre09.g406500,Cre02.g106400",3.0
22009,31v3,M8,2024-09-04,5min-5min,15.602963,21.738926,180,7,11,0.666741,...,NaN,NaN,NaN,NaN,NaN,H12,WT,NaN,"Cre09.g392000,Cre09.g406500,Cre02.g106400",3.0
22144,31v3,M8,2024-09-04,5min-5min,15.602963,21.738926,180,13,2,0.651964,...,NaN,NaN,NaN,NaN,NaN,N03,WT,NaN,"Cre09.g392000,Cre09.g406500,Cre02.g106400",3.0
22153,31v3,M8,2024-09-04,5min-5min,15.602963,21.738926,180,13,11,0.650562,...,NaN,NaN,NaN,NaN,NaN,N12,WT,NaN,"Cre09.g392000,Cre09.g406500,Cre02.g106400",3.0
22163,31v3,M8,2024-09-04,5min-5min,15.602963,21.738926,180,13,21,0.649472,...,NaN,NaN,NaN,NaN,NaN,N22,WT,NaN,"Cre09.g392000,Cre09.g406500,Cre02.g106400",3.0


## 31 plate 20h_ML

In [55]:
def quantile_normalize_light_regime(df, light_regime, plates, y2_cols, tie_handling=True):
    """
    Quantile-normalize all y2_cols across selected plates within a given light regime.
    """
    # Filter data
    subset_df = df[(df['light_regime'] == light_regime) & (df['plate'].isin(plates))].copy()
    df_normalized = subset_df.copy()

    for timepoint in y2_cols:
        position_values = []
        valid_plate_indices = {}

        for plate in plates:
            plate_df = subset_df[subset_df['plate'] == plate].copy()

            wt_rows = plate_df[plate_df['mutant_ID'] == 'WT'].copy()
            non_wt_rows = plate_df[plate_df['mutant_ID'] != 'WT'].copy()

            wt_rows = wt_rows.sort_values(['mutant_ID', 'mutated_genes', 'well_id'])
            non_wt_rows = non_wt_rows.sort_values(['mutant_ID', 'mutated_genes'])

            sorted_df = pd.concat([wt_rows, non_wt_rows], axis=0)
            values = sorted_df[timepoint].values
            index = sorted_df.index.values

            position_values.append(values)
            valid_plate_indices[plate] = index

        # Validate shape
        lengths = [len(v) for v in position_values]
        if len(set(lengths)) != 1:
            raise ValueError(f"Length mismatch at {timepoint}: {lengths}")

        matrix = np.column_stack(position_values)
        normalized_matrix = normalize_quantiles(matrix, ties=tie_handling)

        # Write back
        for col_idx, plate in enumerate(plates):
            indices = valid_plate_indices[plate]
            df_normalized.loc[indices, timepoint] = normalized_matrix[:, col_idx]

    return df_normalized

In [56]:
# Define inputs
plates = ['31v1', '31v2', '31v3']
y2_cols = [f'y2_{i}' for i in range(1, 45)]

# Run normalization
phase2_31_20h_ML_normalized = quantile_normalize_light_regime(
    df=phase2_df1,
    light_regime='20h_ML',
    plates=plates,
    y2_cols=y2_cols
)

# View a few columns
phase2_31_20h_ML_normalized[['plate', 'mutant_ID', 'mutated_genes', 'well_id'] + y2_cols]

,plate,mutant_ID,mutated_genes,well_id,y2_1,y2_2,y2_3,y2_4,y2_5,y2_6,...,y2_35,y2_36,y2_37,y2_38,y2_39,y2_40,y2_41,y2_42,y2_43,y2_44
10724,31v1,LMJ.RY0402.047650,Cre10.g434650,A02,0.468256,0.543581,0.490271,0.484043,0.474957,0.524648,...,0.454527,0.437448,0.499045,0.453701,0.441633,0.470832,0.465403,0.454171,0.474872,0.479754
10725,31v1,LMJ.RY0402.058045,Cre10.g437300,A03,0.376083,0.406178,0.430313,0.313689,0.396032,0.474034,...,0.416964,0.355349,0.352470,0.365407,0.333182,0.353635,0.338907,0.326565,0.330926,0.311753
10726,31v1,LMJ.RY0402.052609,Cre10.g439350,A04,0.624953,0.659758,0.660089,0.661515,0.679083,0.674496,...,0.671539,0.673847,0.669826,0.663388,0.665905,0.669930,0.668022,0.678910,0.670085,0.678834
10727,31v1,LMJ.RY0402.236960,Cre11.g467675,A05,0.410148,0.480815,0.488706,0.474373,0.458904,0.538838,...,0.458188,0.442526,0.449624,0.449593,0.411652,0.457629,0.429265,0.436929,0.430346,0.441415
10728,31v1,LMJ.RY0402.049580,"Cre12.g496100,Cre12.g496150 & Cre12.g496100,Cr...",A06,0.336147,0.338525,0.375852,0.327687,0.337470,0.332204,...,0.291885,0.309545,0.331460,0.291865,0.305408,0.310600,0.287949,0.292753,0.284479,0.311400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25656,31v3,LMJ.RY0402.146100,Cre13.g584901,P20,0.417055,0.437762,0.459412,0.481570,0.444932,0.470107,...,0.434993,0.449368,0.443792,0.431498,0.420317,0.418994,0.439770,0.436123,0.419146,0.387728
25657,31v3,LMJ.RY0402.094443,Cre10.g441850,P21,0.388190,0.360529,0.408298,0.437613,0.423393,0.431554,...,0.396176,0.386839,0.406049,0.426590,0.410067,0.399878,0.372108,0.394057,0.416522,0.377158
25658,31v3,LMJ.RY0402.220921,Cre11.g476050,P22,0.397694,0.427213,0.431023,0.460877,0.407701,0.396615,...,0.405396,0.399217,0.447233,0.449404,0.403150,0.387783,0.407436,0.396756,0.382016,0.417826
25659,31v3,LMJ.RY0402.057187,Cre16.g678050,P23,0.355041,0.385981,0.366610,0.361196,0.394370,0.413991,...,0.338188,0.329693,0.292902,0.326664,0.328097,0.321058,0.310573,0.300591,0.306722,0.319586


In [57]:
plates = ['31v1', '31v2', '31v3']
phase2_31_20h_ML= phase2_df1[(phase2_df1['light_regime'] == '20h_ML') & (phase2_df1['plate'].isin(plates))].copy()
phase2_31_20h_ML[['plate', 'mutant_ID', 'mutated_genes', 'well_id'] + y2_cols]

,plate,mutant_ID,mutated_genes,well_id,y2_1,y2_2,y2_3,y2_4,y2_5,y2_6,...,y2_35,y2_36,y2_37,y2_38,y2_39,y2_40,y2_41,y2_42,y2_43,y2_44
10724,31v1,LMJ.RY0402.047650,Cre10.g434650,A02,0.449257,0.506394,0.472331,0.472022,0.466124,0.505023,...,0.448474,0.437939,0.472576,0.456931,0.442587,0.472372,0.453772,0.453276,0.459875,0.472678
10725,31v1,LMJ.RY0402.058045,Cre10.g437300,A03,0.365389,0.394504,0.414607,0.308445,0.394179,0.462885,...,0.415354,0.364256,0.355441,0.377032,0.339298,0.370396,0.345573,0.338282,0.336992,0.323112
10726,31v1,LMJ.RY0402.052609,Cre10.g439350,A04,0.609689,0.663990,0.647972,0.670877,0.676651,0.679586,...,0.676347,0.685151,0.679814,0.675606,0.674434,0.672508,0.684652,0.691461,0.688581,0.678000
10727,31v1,LMJ.RY0402.236960,Cre11.g467675,A05,0.395750,0.462183,0.470840,0.461421,0.448495,0.511851,...,0.451162,0.440918,0.446825,0.455180,0.418218,0.463333,0.430081,0.442013,0.428636,0.438809
10728,31v1,LMJ.RY0402.049580,"Cre12.g496100,Cre12.g496150 & Cre12.g496100,Cr...",A06,0.326382,0.333585,0.366908,0.322873,0.334471,0.333576,...,0.295898,0.319719,0.335568,0.298858,0.313297,0.327452,0.297869,0.306572,0.290940,0.323044
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25656,31v3,LMJ.RY0402.146100,Cre13.g584901,P20,0.415537,0.431391,0.453878,0.470883,0.437444,0.467190,...,0.426115,0.441382,0.433348,0.416186,0.408916,0.402378,0.428318,0.421154,0.405858,0.373663
25657,31v3,LMJ.RY0402.094443,Cre10.g441850,P21,0.386049,0.357293,0.403901,0.427682,0.414545,0.426936,...,0.381790,0.378833,0.398139,0.412279,0.399266,0.383930,0.354613,0.378185,0.404224,0.360358
25658,31v3,LMJ.RY0402.220921,Cre11.g476050,P22,0.395659,0.422036,0.426888,0.452465,0.399727,0.391448,...,0.393340,0.393079,0.438894,0.431296,0.390728,0.373640,0.391777,0.381441,0.366665,0.403438
25659,31v3,LMJ.RY0402.057187,Cre16.g678050,P23,0.351617,0.382764,0.364827,0.350516,0.383191,0.410381,...,0.326784,0.320409,0.280735,0.310173,0.316747,0.306316,0.297061,0.285890,0.296316,0.305973


## 31 plate 20h_HL

In [58]:
# Define inputs
plates = ['31v1', '31v3']
y2_cols = [f'y2_{i}' for i in range(1, 45)]

# Run normalization
phase2_31_20h_HL_normalized = quantile_normalize_light_regime(
    df=phase2_df1,
    light_regime='20h_HL',
    plates=plates,
    y2_cols=y2_cols
)

# View a few columns
phase2_31_20h_HL_normalized[['plate', 'mutant_ID', 'mutated_genes', 'well_id'] + y2_cols]

,plate,mutant_ID,mutated_genes,well_id,y2_1,y2_2,y2_3,y2_4,y2_5,y2_6,...,y2_35,y2_36,y2_37,y2_38,y2_39,y2_40,y2_41,y2_42,y2_43,y2_44
16469,31v1,LMJ.RY0402.047650,Cre10.g434650,A02,0.329892,0.325980,0.315865,0.331191,0.310608,0.287807,...,0.181950,0.198061,0.191414,0.135485,0.218646,0.145816,0.159419,0.172736,0.154040,0.146566
16470,31v1,LMJ.RY0402.058045,Cre10.g437300,A03,0.173321,0.214454,0.223848,0.210661,0.245879,0.230460,...,0.114796,0.132391,0.170108,0.085774,0.148863,0.141143,0.142146,0.163371,0.157542,0.154542
16471,31v1,LMJ.RY0402.052609,Cre10.g439350,A04,0.417227,0.492758,0.430715,0.465661,0.481897,0.483801,...,0.399237,0.353943,0.381605,0.396384,0.414803,0.421682,0.399230,0.409581,0.378085,0.421646
16472,31v1,LMJ.RY0402.236960,Cre11.g467675,A05,0.201654,0.220247,0.261328,0.260340,0.242796,0.222801,...,0.195636,0.139659,0.189169,0.204011,0.205863,0.173402,0.235033,0.150668,0.156742,0.194531
16473,31v1,LMJ.RY0402.049580,"Cre12.g496100,Cre12.g496150 & Cre12.g496100,Cr...",A06,0.152292,0.191512,0.144459,0.194192,0.159757,0.165706,...,0.100503,0.104199,0.140331,0.066035,0.111724,0.153338,0.114850,0.100313,0.085984,0.089815
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34465,31v3,LMJ.RY0402.146100,Cre13.g584901,P20,0.240135,0.177961,0.191548,0.241467,0.249624,0.259629,...,0.188366,0.178544,0.212470,0.196444,0.181061,0.225915,0.172894,0.189211,0.165677,0.207256
34466,31v3,LMJ.RY0402.094443,Cre10.g441850,P21,0.212083,0.171551,0.213532,0.168396,0.157782,0.157309,...,0.171828,0.186796,0.149472,0.099850,0.160497,0.178695,0.140333,0.161286,0.153100,0.162218
34467,31v3,LMJ.RY0402.220921,Cre11.g476050,P22,0.187546,0.184583,0.185950,0.215065,0.270304,0.252617,...,0.162729,0.143959,0.183413,0.159905,0.128839,0.151337,0.197684,0.141254,0.156736,0.190982
34468,31v3,LMJ.RY0402.057187,Cre16.g678050,P23,0.154991,0.174754,0.179358,0.198580,0.260458,0.209497,...,0.060612,0.109010,0.099969,0.073727,0.123491,0.078436,0.037956,0.063322,0.111541,0.082662


In [59]:
plates = ['31v1', '31v2', '31v3']
phase2_31_20h_HL= phase2_df1[(phase2_df1['light_regime'] == '20h_HL') & (phase2_df1['plate'].isin(plates))].copy()
phase2_31_20h_HL[['plate', 'mutant_ID', 'mutated_genes', 'well_id'] + y2_cols]

,plate,mutant_ID,mutated_genes,well_id,y2_1,y2_2,y2_3,y2_4,y2_5,y2_6,...,y2_35,y2_36,y2_37,y2_38,y2_39,y2_40,y2_41,y2_42,y2_43,y2_44
16469,31v1,LMJ.RY0402.047650,Cre10.g434650,A02,0.319671,0.311774,0.288214,0.309425,0.297522,0.279933,...,0.162893,0.180579,0.181123,0.127619,0.200575,0.139808,0.149378,0.162887,0.149373,0.138723
16470,31v1,LMJ.RY0402.058045,Cre10.g437300,A03,0.178676,0.208460,0.213363,0.202780,0.239000,0.224420,...,0.107790,0.127986,0.162207,0.080478,0.141839,0.135468,0.132288,0.155175,0.152300,0.148111
16471,31v1,LMJ.RY0402.052609,Cre10.g439350,A04,0.405063,0.456915,0.374917,0.410295,0.450445,0.429842,...,0.416392,0.348907,0.383060,0.390298,0.415515,0.425144,0.407641,0.422957,0.377986,0.430786
16472,31v1,LMJ.RY0402.236960,Cre11.g467675,A05,0.205036,0.213706,0.246363,0.249040,0.235669,0.217633,...,0.175658,0.135325,0.177906,0.186906,0.184875,0.166760,0.203495,0.145500,0.151261,0.182958
16473,31v1,LMJ.RY0402.049580,"Cre12.g496100,Cre12.g496150 & Cre12.g496100,Cr...",A06,0.157501,0.185570,0.141948,0.184009,0.157445,0.166169,...,0.093474,0.098130,0.133350,0.064162,0.107450,0.146669,0.106727,0.100834,0.084512,0.088804
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34465,31v3,LMJ.RY0402.146100,Cre13.g584901,P20,0.240721,0.182749,0.199210,0.248268,0.255659,0.268758,...,0.207376,0.188574,0.231208,0.211245,0.197745,0.241831,0.186036,0.202931,0.170786,0.222912
34466,31v3,LMJ.RY0402.094443,Cre10.g441850,P21,0.208607,0.176165,0.222937,0.176413,0.159885,0.155744,...,0.188218,0.201274,0.158795,0.106276,0.169624,0.187339,0.150397,0.167725,0.157826,0.167357
34467,31v3,LMJ.RY0402.220921,Cre11.g476050,P22,0.182458,0.190217,0.193906,0.222442,0.278232,0.262135,...,0.175815,0.148893,0.191595,0.168135,0.136372,0.158647,0.214430,0.144950,0.162156,0.202695
34468,31v3,LMJ.RY0402.057187,Cre16.g678050,P23,0.150670,0.178759,0.185643,0.208744,0.269458,0.211782,...,0.069495,0.113800,0.108997,0.077066,0.129620,0.074179,0.034656,0.057049,0.113595,0.084310


### 31 plate 2h-2h

In [78]:
plates = ['31v1','31v2','31v3']
y2_cols = [f'y2_{i}' for i in range(1, 49)]
# Run normalization
phase2_31_2h_2h_normalized = quantile_normalize_light_regime(
    df=phase2_df1,
    light_regime='2h-2h',
    plates=plates,
    y2_cols=y2_cols
)

phase2_31_2h_2h_normalized[['plate', 'mutant_ID', 'mutated_genes', 'well_id'] + y2_cols]

,plate,mutant_ID,mutated_genes,well_id,y2_1,y2_2,y2_3,y2_4,y2_5,y2_6,...,y2_39,y2_40,y2_41,y2_42,y2_43,y2_44,y2_45,y2_46,y2_47,y2_48
9575,31v1,LMJ.RY0402.047650,Cre10.g434650,A02,0.256668,0.355772,0.265001,0.306589,0.353794,0.276044,...,0.691554,0.686843,0.232144,0.230416,0.284128,0.250139,0.700161,0.701201,0.681841,0.683821
9576,31v1,LMJ.RY0402.058045,Cre10.g437300,A03,0.217917,0.208267,0.167179,0.215877,0.164968,0.236115,...,0.629687,0.593548,0.145040,0.194588,0.255536,0.200496,0.598547,0.600967,0.601203,0.607519
9577,31v1,LMJ.RY0402.052609,Cre10.g439350,A04,0.479022,0.532894,0.432549,0.525330,0.502126,0.495123,...,0.758764,0.750311,0.405004,0.470981,0.467760,0.444504,0.742472,0.754769,0.746201,0.752965
9578,31v1,LMJ.RY0402.236960,Cre11.g467675,A05,0.258456,0.216825,0.184218,0.234180,0.226765,0.258911,...,0.688336,0.667391,0.215808,0.243376,0.269606,0.234636,0.683073,0.690429,0.674179,0.687132
9579,31v1,LMJ.RY0402.049580,"Cre12.g496100,Cre12.g496150 & Cre12.g496100,Cr...",A06,0.184153,0.204554,0.165353,0.212457,0.185600,0.169805,...,0.586177,0.564585,0.161173,0.127843,0.167449,0.137552,0.551866,0.558163,0.562188,0.561918
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30252,31v2,LMJ.RY0402.159750,"Cre07.g339104,Cre06.g303200",P20,0.171238,0.178737,0.191297,0.231849,0.181672,0.184189,...,0.577730,0.582925,0.120613,0.199896,0.201508,0.112547,0.560324,0.562038,0.575637,0.574505
30253,31v2,LMJ.RY0402.121674,Cre16.g685250,P21,0.140517,0.214997,0.167194,0.185518,0.100685,0.075921,...,0.520409,0.556265,0.150914,0.116272,0.113098,0.144251,0.521174,0.495247,0.533049,0.544919
30254,31v2,LMJ.RY0402.191617,Cre09.g401960,P22,0.188491,0.145768,0.153660,0.186246,0.137253,0.132266,...,0.546741,0.559235,0.111736,0.107373,0.078213,0.184727,0.553745,0.520724,0.550476,0.551576
30255,31v2,LMJ.RY0402.076124,Cre14.g627576,P23,0.160920,0.154045,0.222430,0.224510,0.171297,0.172057,...,0.594597,0.578104,0.113104,0.180332,0.158296,0.186400,0.571536,0.576135,0.571071,0.585254


In [79]:

plates = ['31v1', '31v2', '31v3']
phase2_31_2h_2h = phase2_df1[
    (phase2_df1['light_regime'] == '2h-2h') &
    (phase2_df1['plate'].isin(plates))
].copy()
phase2_31_2h_2h[['plate', 'mutant_ID', 'mutated_genes', 'well_id'] + y2_columns]

,plate,mutant_ID,mutated_genes,well_id,y2_1,y2_2,y2_3,y2_4,y2_5,y2_6,...,y2_75,y2_76,y2_77,y2_78,y2_79,y2_80,y2_81,y2_82,y2_83,y2_84
9575,31v1,LMJ.RY0402.047650,Cre10.g434650,A02,0.265284,0.337608,0.296833,0.334255,0.338882,0.298534,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9576,31v1,LMJ.RY0402.058045,Cre10.g437300,A03,0.226415,0.227517,0.194254,0.237077,0.185025,0.256351,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9577,31v1,LMJ.RY0402.052609,Cre10.g439350,A04,0.431015,0.495661,0.483352,0.490876,0.457430,0.489414,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9578,31v1,LMJ.RY0402.236960,Cre11.g467675,A05,0.266800,0.235242,0.214238,0.255597,0.259276,0.282088,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9579,31v1,LMJ.RY0402.049580,"Cre12.g496100,Cre12.g496150 & Cre12.g496100,Cr...",A06,0.192247,0.224556,0.191742,0.232683,0.209450,0.184807,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30252,31v2,LMJ.RY0402.159750,"Cre07.g339104,Cre06.g303200",P20,0.168966,0.162366,0.173700,0.201965,0.152637,0.162705,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30253,31v2,LMJ.RY0402.121674,Cre16.g685250,P21,0.140090,0.196162,0.151416,0.158976,0.096321,0.064695,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30254,31v2,LMJ.RY0402.191617,Cre09.g401960,P22,0.184811,0.135582,0.140513,0.159466,0.111736,0.118211,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30255,31v2,LMJ.RY0402.076124,Cre14.g627576,P23,0.159508,0.141909,0.204572,0.195852,0.144184,0.150868,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 31 plate 10min-10min

In [63]:
plates = ['31v2','31v3']
y2_cols = [f'y2_{i}' for i in range(1, 85)]

# phase2_31_10min_10min_normalized = quantile_normalize_plate_group(
#     df=phase2_df1,
#     light_regime='10min-10min',
#     plates=plates_31,
#     y2_cols=y2_columns,
#     wt_n_replicates=21
# )
phase2_31_10min_10min_normalized = quantile_normalize_light_regime(
    df=phase2_df1,
    light_regime='10min-10min',
    plates=plates,
    y2_cols=y2_cols
)


phase2_31_10min_10min_normalized[['plate', 'mutant_ID', 'mutated_genes', 'well_id'] + y2_cols]

,plate,mutant_ID,mutated_genes,well_id,y2_1,y2_2,y2_3,y2_4,y2_5,y2_6,...,y2_75,y2_76,y2_77,y2_78,y2_79,y2_80,y2_81,y2_82,y2_83,y2_84
6128,31v3,LMJ.RY0402.051628,"Cre13.g589400,Cre13.g589350",A02,0.110967,0.551987,0.174808,0.536180,0.110798,0.547420,...,0.174868,0.545732,0.548342,0.168603,0.147750,0.535721,0.547379,0.135399,0.109597,0.557851
6129,31v3,LMJ.RY0402.189077,Cre08.g384320,A03,0.245996,0.508059,0.043704,0.507586,0.134984,0.525783,...,0.115979,0.506883,0.481554,0.051502,0.054263,0.479114,0.496414,0.135633,0.126743,0.538553
6130,31v3,LMJ.RY0402.251390,Cre13.g581050,A04,0.135484,0.551376,0.176741,0.512074,0.115909,0.524506,...,0.130889,0.511713,0.501993,0.150296,0.120863,0.523250,0.498579,0.111431,0.108447,0.496806
6131,31v3,LMJ.RY0402.214268,Cre09.g400849,A05,0.161034,0.527220,0.136473,0.520368,0.195507,0.511091,...,0.102295,0.516280,0.532130,0.157773,0.108288,0.505855,0.517415,0.076600,0.119632,0.516229
6132,31v3,LMJ.RY0402.068861,Cre16.g664400,A06,0.167651,0.576752,0.180733,0.566796,0.172785,0.565596,...,0.148181,0.531568,0.527776,0.113787,0.147481,0.561776,0.537387,0.162529,0.136827,0.560618
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36380,31v2,LMJ.RY0402.159750,"Cre07.g339104,Cre06.g303200",P20,0.143199,0.537465,0.197111,0.547578,0.143031,0.540935,...,0.085372,0.532690,0.542943,0.137518,0.112208,0.542417,0.562598,0.123841,0.114033,0.532194
36381,31v2,LMJ.RY0402.121674,Cre16.g685250,P21,0.088142,0.464568,0.107375,0.466412,0.079056,0.511681,...,0.081433,0.496331,0.523701,0.137965,0.131057,0.454373,0.461479,0.069085,0.089325,0.481549
36382,31v2,LMJ.RY0402.191617,Cre09.g401960,P22,0.184094,0.522155,0.197268,0.511167,0.121631,0.534778,...,0.091627,0.512339,0.521748,0.128129,0.168040,0.529302,0.551425,0.117403,0.144782,0.516107
36383,31v2,LMJ.RY0402.076124,Cre14.g627576,P23,0.207451,0.544110,0.129272,0.517785,0.100401,0.471437,...,0.132709,0.551514,0.555463,0.160225,0.118384,0.532916,0.569216,0.120761,0.147258,0.539604


In [64]:

plates = [ '31v2', '31v3']
phase2_31_10min_10min = phase2_df1[
    (phase2_df1['light_regime'] == '10min-10min') &
    (phase2_df1['plate'].isin(plates))
].copy()
phase2_31_10min_10min[['plate', 'mutant_ID', 'mutated_genes', 'well_id'] + y2_cols ]

,plate,mutant_ID,mutated_genes,well_id,y2_1,y2_2,y2_3,y2_4,y2_5,y2_6,...,y2_75,y2_76,y2_77,y2_78,y2_79,y2_80,y2_81,y2_82,y2_83,y2_84
6128,31v3,LMJ.RY0402.051628,"Cre13.g589400,Cre13.g589350",A02,0.104398,0.546321,0.177219,0.530902,0.115949,0.541055,...,0.175546,0.544950,0.546405,0.171568,0.155526,0.533819,0.549485,0.147029,0.118359,0.556407
6129,31v3,LMJ.RY0402.189077,Cre08.g384320,A03,0.242582,0.496405,0.029543,0.500642,0.140968,0.517353,...,0.117063,0.503213,0.469805,0.051023,0.058993,0.477368,0.491838,0.147134,0.135985,0.534309
6130,31v3,LMJ.RY0402.251390,Cre13.g581050,A04,0.128410,0.545449,0.178662,0.505112,0.123043,0.516023,...,0.128885,0.509808,0.495457,0.152724,0.128030,0.517284,0.496725,0.122982,0.117335,0.490880
6131,31v3,LMJ.RY0402.214268,Cre09.g400849,A05,0.157017,0.521018,0.136084,0.513692,0.204371,0.503081,...,0.101973,0.513771,0.529268,0.160058,0.115452,0.503953,0.515756,0.086030,0.127384,0.513132
6132,31v3,LMJ.RY0402.068861,Cre16.g664400,A06,0.164113,0.569538,0.181776,0.562649,0.179403,0.559855,...,0.146150,0.530788,0.525288,0.116261,0.155289,0.562307,0.539640,0.174964,0.146767,0.559393
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36380,31v2,LMJ.RY0402.159750,"Cre07.g339104,Cre06.g303200",P20,0.149971,0.545557,0.195598,0.553673,0.138577,0.548304,...,0.085703,0.533498,0.544602,0.134023,0.104376,0.543382,0.561461,0.111974,0.104485,0.537424
36381,31v2,LMJ.RY0402.121674,Cre16.g685250,P21,0.102475,0.469612,0.113887,0.487517,0.082856,0.520301,...,0.081894,0.500525,0.526651,0.134381,0.122391,0.464754,0.475404,0.060718,0.081659,0.488189
36382,31v2,LMJ.RY0402.191617,Cre09.g401960,P22,0.186011,0.526815,0.195678,0.518297,0.117729,0.542464,...,0.090334,0.514575,0.524852,0.124798,0.161334,0.533673,0.549268,0.104160,0.133787,0.519096
36383,31v2,LMJ.RY0402.076124,Cre14.g627576,P23,0.209269,0.550546,0.132420,0.524220,0.098018,0.483949,...,0.134869,0.552262,0.557603,0.158200,0.110347,0.535513,0.566455,0.108580,0.136567,0.543590


## 31 plate 1min-1min

In [65]:
plates = ['31v1','31v2','31v3']
y2_cols = [f'y2_{i}' for i in range(1, 89)]


phase2_31_1min_1min_normalized = quantile_normalize_light_regime(
    df=phase2_df1,
    light_regime='1min-1min',
    plates=plates,
    y2_cols=y2_cols
)



phase2_31_1min_1min_normalized[['plate', 'mutant_ID', 'mutated_genes', 'well_id'] + y2_cols]

,plate,mutant_ID,mutated_genes,well_id,y2_1,y2_2,y2_3,y2_4,y2_5,y2_6,...,y2_79,y2_80,y2_81,y2_82,y2_83,y2_84,y2_85,y2_86,y2_87,y2_88
13022,31v2,LMJ.RY0402.219500,"Cre13.g587850,Cre04.g216850",A02,0.332028,0.643181,0.315059,0.620355,0.282112,0.623903,...,0.253839,0.615414,0.251954,0.612626,0.289182,0.593666,0.258215,0.610348,0.272618,0.610082
13023,31v2,LMJ.RY0402.130430,Cre06.g303200,A03,0.241061,0.587635,0.230942,0.563395,0.196370,0.530341,...,0.211288,0.531482,0.170294,0.537559,0.193307,0.527269,0.142025,0.500322,0.190597,0.539867
13024,31v2,LMJ.RY0402.057187,Cre16.g678050,A04,0.236567,0.567938,0.164078,0.531562,0.256578,0.526442,...,0.176803,0.501071,0.171415,0.516816,0.195424,0.482374,0.174685,0.515568,0.146570,0.542736
13025,31v2,LMJ.RY0402.200883,"Cre09.g404503,Cre09.g404500 & Cre09.g404503",A05,0.165501,0.505924,0.165433,0.505043,0.178468,0.472690,...,0.167201,0.510551,0.112671,0.465308,0.166711,0.478769,0.129803,0.492155,0.139802,0.490720
13026,31v2,LMJ.RY0402.127527,Cre08.g384320,A06,0.183464,0.534611,0.178192,0.518321,0.219493,0.502329,...,0.199351,0.474247,0.154760,0.502978,0.171884,0.462318,0.135666,0.478530,0.162464,0.493436
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33316,31v3,LMJ.RY0402.146100,Cre13.g584901,P20,0.238700,0.581222,0.251222,0.554783,0.251198,0.583197,...,0.233219,0.568884,0.247303,0.545497,0.270313,0.558816,0.187607,0.552135,0.244176,0.520388
33317,31v3,LMJ.RY0402.094443,Cre10.g441850,P21,0.202550,0.534700,0.197656,0.477877,0.222282,0.513160,...,0.116493,0.496573,0.177653,0.496391,0.178278,0.458057,0.230025,0.494669,0.215438,0.507704
33318,31v3,LMJ.RY0402.220921,Cre11.g476050,P22,0.228787,0.505159,0.273550,0.545052,0.238210,0.541337,...,0.205840,0.506341,0.204469,0.508407,0.226623,0.562660,0.211393,0.553065,0.172616,0.573795
33319,31v3,LMJ.RY0402.057187,Cre16.g678050,P23,0.172337,0.543404,0.198706,0.560067,0.189072,0.545183,...,0.140418,0.458319,0.166926,0.460102,0.174303,0.497961,0.062395,0.523232,0.088005,0.471164


In [66]:

plates = ['31v1', '31v2', '31v3']
phase2_31_1min_1min = phase2_df1[
    (phase2_df1['light_regime'] == '1min-1min') &
    (phase2_df1['plate'].isin(plates))
].copy()
phase2_31_1min_1min[['plate', 'mutant_ID', 'mutated_genes', 'well_id'] + y2_cols ]

,plate,mutant_ID,mutated_genes,well_id,y2_1,y2_2,y2_3,y2_4,y2_5,y2_6,...,y2_79,y2_80,y2_81,y2_82,y2_83,y2_84,y2_85,y2_86,y2_87,y2_88
13022,31v2,LMJ.RY0402.219500,"Cre13.g587850,Cre04.g216850",A02,0.339646,0.654450,0.321424,0.640876,0.288198,0.630318,...,0.262269,0.616402,0.261060,0.621992,0.303215,0.608281,0.269029,0.618456,0.275982,0.615127
13023,31v2,LMJ.RY0402.130430,Cre06.g303200,A03,0.252636,0.591293,0.229160,0.569196,0.202126,0.527699,...,0.213290,0.532155,0.179016,0.543922,0.203109,0.537394,0.154785,0.499441,0.196644,0.547487
13024,31v2,LMJ.RY0402.057187,Cre16.g678050,A04,0.248526,0.572415,0.167403,0.534326,0.262744,0.523830,...,0.175952,0.498672,0.179753,0.523781,0.205026,0.489986,0.186673,0.516289,0.152296,0.549344
13025,31v2,LMJ.RY0402.200883,"Cre09.g404503,Cre09.g404500 & Cre09.g404503",A05,0.176564,0.509879,0.168150,0.508949,0.185749,0.476151,...,0.165961,0.509065,0.124952,0.472942,0.175248,0.487128,0.145449,0.490816,0.144752,0.491246
13026,31v2,LMJ.RY0402.127527,Cre08.g384320,A06,0.191978,0.538939,0.180410,0.521354,0.224673,0.499247,...,0.201673,0.473060,0.164807,0.507158,0.180919,0.473593,0.148827,0.478280,0.171066,0.494783
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33316,31v3,LMJ.RY0402.146100,Cre13.g584901,P20,0.229399,0.576796,0.247700,0.550800,0.254829,0.585044,...,0.214754,0.563971,0.232688,0.530670,0.254304,0.548128,0.168582,0.545124,0.230153,0.510508
33317,31v3,LMJ.RY0402.094443,Cre10.g441850,P21,0.195202,0.530701,0.197225,0.474758,0.222248,0.516934,...,0.100951,0.495793,0.164770,0.484224,0.160955,0.446801,0.208930,0.487567,0.199871,0.499141
33318,31v3,LMJ.RY0402.220921,Cre11.g476050,P22,0.218505,0.500699,0.269919,0.542150,0.236534,0.547652,...,0.191429,0.506132,0.191315,0.496103,0.207963,0.552545,0.192432,0.546825,0.158429,0.561611
33319,31v3,LMJ.RY0402.057187,Cre16.g678050,P23,0.165221,0.541435,0.198429,0.555779,0.189663,0.550856,...,0.125039,0.457756,0.153553,0.443806,0.156601,0.491684,0.033490,0.515071,0.064629,0.464831


## 31 plate 30s-30s

In [67]:
plates = ['31v1','31v2','31v3']
y2_cols = [f'y2_{i}' for i in range(1, 89)]


phase2_31_30s_30s_normalized = quantile_normalize_light_regime(
    df=phase2_df1,
    light_regime='30s-30s',
    plates=plates,
    y2_cols=y2_cols
)



phase2_31_30s_30s_normalized[['plate', 'mutant_ID', 'mutated_genes', 'well_id'] + y2_cols]

,plate,mutant_ID,mutated_genes,well_id,y2_1,y2_2,y2_3,y2_4,y2_5,y2_6,...,y2_79,y2_80,y2_81,y2_82,y2_83,y2_84,y2_85,y2_86,y2_87,y2_88
11490,31v1,LMJ.RY0402.047650,Cre10.g434650,A02,0.324264,0.616195,0.303719,0.577918,0.312570,0.586258,...,0.256818,0.593888,0.321403,0.566479,0.275276,0.574299,0.274891,0.607533,0.299491,0.531041
11491,31v1,LMJ.RY0402.058045,Cre10.g437300,A03,0.232603,0.532869,0.213831,0.531256,0.200933,0.501867,...,0.226282,0.495179,0.219076,0.508122,0.205343,0.492477,0.228805,0.493319,0.223642,0.505262
11492,31v1,LMJ.RY0402.052609,Cre10.g439350,A04,0.365319,0.723857,0.428611,0.695671,0.430257,0.716081,...,0.357675,0.655626,0.387639,0.663236,0.379702,0.690293,0.396914,0.673917,0.404196,0.649390
11493,31v1,LMJ.RY0402.236960,Cre11.g467675,A05,0.296067,0.636793,0.288086,0.571769,0.296927,0.594429,...,0.312899,0.568297,0.227363,0.561895,0.264183,0.576400,0.272652,0.586488,0.300845,0.569967
11494,31v1,LMJ.RY0402.049580,"Cre12.g496100,Cre12.g496150 & Cre12.g496100,Cr...",A06,0.159939,0.516604,0.147782,0.505116,0.162139,0.501176,...,0.117205,0.483381,0.115221,0.469079,0.169457,0.449540,0.143662,0.451342,0.183031,0.482186
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23358,31v2,LMJ.RY0402.159750,"Cre07.g339104,Cre06.g303200",P20,0.200442,0.536929,0.212923,0.520607,0.174442,0.513690,...,0.186932,0.455493,0.183350,0.447403,0.148031,0.488835,0.166110,0.509607,0.155713,0.470380
23359,31v2,LMJ.RY0402.121674,Cre16.g685250,P21,0.147657,0.488097,0.126633,0.463305,0.166729,0.483635,...,0.146602,0.437923,0.142726,0.427936,0.106571,0.464013,0.124681,0.442465,0.113393,0.438292
23360,31v2,LMJ.RY0402.191617,Cre09.g401960,P22,0.177231,0.506233,0.096158,0.505765,0.193006,0.464607,...,0.143688,0.444621,0.171159,0.466787,0.069294,0.443784,0.183018,0.459600,0.143070,0.434259
23361,31v2,LMJ.RY0402.076124,Cre14.g627576,P23,0.214013,0.528665,0.211387,0.552297,0.215551,0.541001,...,0.155736,0.482361,0.168007,0.460694,0.215717,0.500675,0.202175,0.476338,0.183941,0.493978


In [68]:

plates = ['31v1', '31v2', '31v3']
phase2_31_30s_30s = phase2_df1[
    (phase2_df1['light_regime'] == '30s-30s') &
    (phase2_df1['plate'].isin(plates))
].copy()
phase2_31_30s_30s[['plate', 'mutant_ID', 'mutated_genes', 'well_id'] + y2_cols ]

,plate,mutant_ID,mutated_genes,well_id,y2_1,y2_2,y2_3,y2_4,y2_5,y2_6,...,y2_79,y2_80,y2_81,y2_82,y2_83,y2_84,y2_85,y2_86,y2_87,y2_88
11490,31v1,LMJ.RY0402.047650,Cre10.g434650,A02,0.340101,0.624521,0.319202,0.589191,0.325900,0.593094,...,0.280107,0.604463,0.339402,0.575791,0.287580,0.583398,0.290804,0.612356,0.303712,0.537946
11491,31v1,LMJ.RY0402.058045,Cre10.g437300,A03,0.245580,0.538954,0.230285,0.538902,0.215305,0.510216,...,0.250098,0.512381,0.237173,0.519294,0.219599,0.502637,0.240192,0.502086,0.232780,0.515128
11492,31v1,LMJ.RY0402.052609,Cre10.g439350,A04,0.372884,0.715632,0.435912,0.695397,0.448710,0.711555,...,0.391742,0.682582,0.420854,0.670449,0.394247,0.697293,0.409957,0.686785,0.412994,0.664765
11493,31v1,LMJ.RY0402.236960,Cre11.g467675,A05,0.307090,0.639393,0.302675,0.582400,0.314604,0.598882,...,0.327958,0.583291,0.243361,0.572811,0.277776,0.585215,0.287615,0.595944,0.304731,0.573273
11494,31v1,LMJ.RY0402.049580,"Cre12.g496100,Cre12.g496150 & Cre12.g496100,Cr...",A06,0.171394,0.525099,0.156687,0.512275,0.175810,0.509492,...,0.136873,0.501909,0.126053,0.478130,0.184945,0.459943,0.151393,0.457950,0.190827,0.487720
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23358,31v2,LMJ.RY0402.159750,"Cre07.g339104,Cre06.g303200",P20,0.202526,0.534663,0.215832,0.522324,0.179062,0.515672,...,0.179206,0.446366,0.184593,0.445982,0.146360,0.486436,0.168029,0.514123,0.161646,0.470738
23359,31v2,LMJ.RY0402.121674,Cre16.g685250,P21,0.157065,0.488226,0.140312,0.465949,0.172703,0.484746,...,0.142834,0.423665,0.145781,0.428677,0.108896,0.461898,0.134317,0.446302,0.120071,0.437612
23360,31v2,LMJ.RY0402.191617,Cre09.g401960,P22,0.181086,0.505820,0.116290,0.505981,0.197268,0.465646,...,0.140946,0.434406,0.171266,0.465962,0.079404,0.444294,0.180503,0.466054,0.151089,0.434804
23361,31v2,LMJ.RY0402.076124,Cre14.g627576,P23,0.216229,0.528424,0.214279,0.556107,0.216793,0.544436,...,0.150068,0.471279,0.168664,0.457435,0.211688,0.498078,0.203212,0.484665,0.190807,0.492602


## 31 plate 5min-5min

In [69]:
plates = ['31v1','31v2','31v3']
y2_cols = [f'y2_{i}' for i in range(1, 89)]


phase2_31_5min_5min_normalized = quantile_normalize_light_regime(
    df=phase2_df1,
    light_regime='5min-5min',
    plates=plates,
    y2_cols=y2_cols
)



phase2_31_5min_5min_normalized[['plate', 'mutant_ID', 'mutated_genes', 'well_id'] + y2_cols]

,plate,mutant_ID,mutated_genes,well_id,y2_1,y2_2,y2_3,y2_4,y2_5,y2_6,...,y2_79,y2_80,y2_81,y2_82,y2_83,y2_84,y2_85,y2_86,y2_87,y2_88
18384,31v2,LMJ.RY0402.219500,"Cre13.g587850,Cre04.g216850",A02,0.282561,0.688739,0.267487,0.660973,0.275179,0.655176,...,0.204784,0.640586,0.236447,0.648950,0.185683,0.623667,0.187257,0.634393,0.172843,0.623739
18385,31v2,LMJ.RY0402.130430,Cre06.g303200,A03,0.241576,0.631901,0.207444,0.614304,0.200717,0.616482,...,0.191248,0.582977,0.182900,0.589258,0.162974,0.580949,0.162535,0.588593,0.181553,0.593209
18386,31v2,LMJ.RY0402.057187,Cre16.g678050,A04,0.166580,0.577642,0.155032,0.555616,0.188161,0.558189,...,0.145094,0.537488,0.112580,0.531175,0.137714,0.534475,0.099418,0.546335,0.126217,0.525557
18387,31v2,LMJ.RY0402.200883,"Cre09.g404503,Cre09.g404500 & Cre09.g404503",A05,0.121042,0.528001,0.150750,0.526689,0.073183,0.533426,...,0.055150,0.494219,0.069506,0.497568,0.058017,0.502537,0.057498,0.506088,0.085573,0.505590
18388,31v2,LMJ.RY0402.127527,Cre08.g384320,A06,0.174812,0.546864,0.167670,0.533470,0.136414,0.532636,...,0.085621,0.494551,0.061293,0.506511,0.121565,0.497783,0.095139,0.506523,0.132662,0.484744
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32933,31v1,NaN,NaN,P20,0.109017,0.644565,0.156194,0.663959,0.175299,0.681840,...,0.161955,0.645821,0.127269,0.576569,0.183471,0.612287,0.150268,0.620813,0.204984,0.608733
32934,31v1,NaN,NaN,P21,0.169173,0.711470,0.213706,0.706691,0.211584,0.655111,...,0.112010,0.619597,0.212632,0.649956,0.169797,0.621892,0.202500,0.663241,0.195349,0.594646
32935,31v1,NaN,NaN,P22,0.209337,0.669687,0.260980,0.656624,0.200906,0.674886,...,0.375161,0.650486,0.247323,0.681601,0.312862,0.686092,0.209839,0.681885,0.313274,0.667823
32936,31v1,NaN,NaN,P23,0.474317,0.738543,0.258832,0.720165,0.454595,0.705422,...,0.396364,0.702795,0.393021,0.569445,0.209021,0.694013,0.111436,0.698403,0.395866,0.704663


In [70]:

plates = ['31v1', '31v2', '31v3']
phase2_31_5min_5min = phase2_df1[
    (phase2_df1['light_regime'] == '5min-5min') &
    (phase2_df1['plate'].isin(plates))
].copy()
phase2_31_5min_5min[['plate', 'mutant_ID', 'mutated_genes', 'well_id'] + y2_cols ]

,plate,mutant_ID,mutated_genes,well_id,y2_1,y2_2,y2_3,y2_4,y2_5,y2_6,...,y2_79,y2_80,y2_81,y2_82,y2_83,y2_84,y2_85,y2_86,y2_87,y2_88
18384,31v2,LMJ.RY0402.219500,"Cre13.g587850,Cre04.g216850",A02,0.319229,0.703396,0.285115,0.677657,0.293690,0.668190,...,0.226199,0.656083,0.267779,0.667621,0.214718,0.637394,0.207658,0.650571,0.192214,0.640419
18385,31v2,LMJ.RY0402.130430,Cre06.g303200,A03,0.267403,0.633068,0.208676,0.613611,0.203866,0.617397,...,0.211095,0.581774,0.205606,0.596477,0.184878,0.585337,0.177521,0.597187,0.202165,0.603170
18386,31v2,LMJ.RY0402.057187,Cre16.g678050,A04,0.173123,0.569054,0.151141,0.543835,0.185694,0.545794,...,0.153446,0.522953,0.122061,0.523479,0.151164,0.525532,0.107438,0.539538,0.132947,0.519024
18387,31v2,LMJ.RY0402.200883,"Cre09.g404503,Cre09.g404500 & Cre09.g404503",A05,0.119082,0.515446,0.146741,0.513554,0.072529,0.514837,...,0.065665,0.475351,0.078378,0.486146,0.067777,0.489845,0.066870,0.494577,0.089114,0.496726
18388,31v2,LMJ.RY0402.127527,Cre08.g384320,A06,0.182598,0.532575,0.164569,0.521232,0.128032,0.514728,...,0.086238,0.475526,0.071462,0.498068,0.133361,0.484076,0.100898,0.494784,0.140701,0.479185
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32933,31v1,NaN,NaN,P20,0.114882,0.642890,0.161466,0.640121,0.178455,0.647446,...,0.132628,0.617657,0.103537,0.578313,0.143823,0.595500,0.122170,0.602134,0.156245,0.595240
32934,31v1,NaN,NaN,P21,0.165238,0.667861,0.204486,0.656585,0.204285,0.635737,...,0.092509,0.605062,0.156500,0.613618,0.135537,0.602057,0.153630,0.623426,0.150972,0.587903
32935,31v1,NaN,NaN,P22,0.190437,0.652577,0.235079,0.634652,0.195951,0.641358,...,0.223930,0.621995,0.178882,0.630143,0.199353,0.628897,0.157966,0.633107,0.201696,0.617803
32936,31v1,NaN,NaN,P23,0.316905,0.695669,0.233575,0.667459,0.288337,0.652882,...,0.248447,0.655899,0.218367,0.574468,0.160804,0.634085,0.091396,0.644213,0.252259,0.651414


## 31 plate 1min-5min¶

In [71]:
plates = ['31v1','31v2','31v3']
y2_cols = [f'y2_{i}' for i in range(1, 89)]


phase2_31_1min_5min_normalized = quantile_normalize_light_regime(
    df=phase2_df1,
    light_regime='1min-5min',
    plates=plates,
    y2_cols=y2_cols
)



phase2_31_1min_5min_normalized[['plate', 'mutant_ID', 'mutated_genes', 'well_id'] + y2_cols]

,plate,mutant_ID,mutated_genes,well_id,y2_1,y2_2,y2_3,y2_4,y2_5,y2_6,...,y2_79,y2_80,y2_81,y2_82,y2_83,y2_84,y2_85,y2_86,y2_87,y2_88
22214,31v3,LMJ.RY0402.051628,"Cre13.g589400,Cre13.g589350",A02,0.132154,0.523796,0.120516,0.528458,0.159443,0.504013,...,0.097950,0.549140,0.105522,0.539846,0.092937,0.546847,0.051109,0.530220,0.105000,0.542416
22215,31v3,LMJ.RY0402.189077,Cre08.g384320,A03,0.178481,0.539482,0.155899,0.534245,0.115229,0.547539,...,0.127580,0.527279,0.103363,0.538548,0.130058,0.533880,0.096815,0.543537,0.129161,0.522696
22216,31v3,LMJ.RY0402.251390,Cre13.g581050,A04,0.097445,0.497623,0.097397,0.494350,0.089643,0.511897,...,0.051508,0.447348,0.060655,0.467128,0.051059,0.513116,0.038535,0.492671,0.040847,0.487678
22217,31v3,LMJ.RY0402.214268,Cre09.g400849,A05,0.123313,0.518399,0.044894,0.483315,0.023104,0.498954,...,0.080404,0.505647,0.028208,0.495449,0.080656,0.519195,0.069902,0.495210,0.056149,0.479149
22218,31v3,LMJ.RY0402.068861,Cre16.g664400,A06,0.201748,0.581422,0.193057,0.540280,0.172870,0.547874,...,0.112523,0.560095,0.133019,0.555209,0.125757,0.537159,0.114482,0.554415,0.131389,0.544874
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37529,31v1,NaN,NaN,P20,0.000486,0.744200,0.044921,0.759685,0.098551,0.757214,...,0.049505,0.656887,0.036938,0.678384,0.057723,0.659206,0.021084,0.690478,0.054460,0.667983
37530,31v1,NaN,NaN,P21,0.079767,0.753203,0.077813,0.747018,0.088417,0.741278,...,0.080520,0.685685,0.075746,0.687468,0.027259,0.686836,0.042577,0.662518,0.038367,0.666071
37531,31v1,NaN,NaN,P22,0.439974,0.751800,0.361636,0.744967,0.386749,0.730140,...,0.343528,0.739542,0.339897,0.722662,0.347117,0.734337,0.332286,0.728255,0.315896,0.723693
37532,31v1,NaN,NaN,P23,0.383400,0.726568,0.433081,0.661459,0.362672,0.744090,...,0.368808,0.707697,0.288552,0.762996,0.254195,0.745413,0.318743,0.754433,0.341811,0.753195


In [72]:

plates = ['31v1', '31v2', '31v3']
phase2_31_1min_5min = phase2_df1[
    (phase2_df1['light_regime'] == '1min-5min') &
    (phase2_df1['plate'].isin(plates))
].copy()
phase2_31_1min_5min[['plate', 'mutant_ID', 'mutated_genes', 'well_id'] + y2_cols ]

,plate,mutant_ID,mutated_genes,well_id,y2_1,y2_2,y2_3,y2_4,y2_5,y2_6,...,y2_79,y2_80,y2_81,y2_82,y2_83,y2_84,y2_85,y2_86,y2_87,y2_88
22214,31v3,LMJ.RY0402.051628,"Cre13.g589400,Cre13.g589350",A02,0.144263,0.525829,0.137365,0.534703,0.176562,0.510130,...,0.111124,0.556725,0.114787,0.544758,0.101398,0.555820,0.058203,0.534891,0.121422,0.548937
22215,31v3,LMJ.RY0402.189077,Cre08.g384320,A03,0.183883,0.539223,0.172975,0.538353,0.131580,0.552092,...,0.138667,0.539042,0.112222,0.542922,0.139871,0.539795,0.108871,0.546374,0.144435,0.527595
22216,31v3,LMJ.RY0402.251390,Cre13.g581050,A04,0.102653,0.506253,0.110872,0.494855,0.101101,0.516618,...,0.055198,0.471159,0.068906,0.484090,0.060854,0.525531,0.044132,0.493982,0.048557,0.496335
22217,31v3,LMJ.RY0402.214268,Cre09.g400849,A05,0.131789,0.521373,0.063945,0.492748,0.061572,0.506089,...,0.092284,0.517058,0.032589,0.505693,0.093134,0.527805,0.082955,0.498532,0.072113,0.489363
22218,31v3,LMJ.RY0402.068861,Cre16.g664400,A06,0.209654,0.579868,0.209920,0.548015,0.190342,0.552201,...,0.124538,0.569585,0.143013,0.562533,0.135880,0.543067,0.128146,0.557844,0.147330,0.551394
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37529,31v1,NaN,NaN,P20,-0.053483,0.750330,0.027773,0.761961,0.087269,0.763721,...,0.047578,0.654008,0.042303,0.673126,0.050149,0.653342,0.023326,0.690925,0.041647,0.666226
37530,31v1,NaN,NaN,P21,0.070646,0.758831,0.067907,0.751728,0.081313,0.750790,...,0.074645,0.676201,0.073664,0.684308,0.026093,0.680614,0.044151,0.660932,0.028976,0.665832
37531,31v1,NaN,NaN,P22,0.448973,0.757829,0.355855,0.745971,0.385851,0.741206,...,0.358959,0.740266,0.338379,0.735368,0.336448,0.732599,0.331400,0.737763,0.302244,0.732226
37532,31v1,NaN,NaN,P23,0.376730,0.730084,0.422836,0.659086,0.353364,0.753827,...,0.360584,0.715853,0.283009,0.766378,0.249639,0.743168,0.330081,0.750556,0.359299,0.755271


In [80]:
phase2_31_quantile1= pd.concat([
    phase2_31_20h_ML_normalized,
    phase2_31_20h_HL_normalized,
    phase2_31_2h_2h_normalized,
    phase2_31_10min_10min_normalized,
    phase2_31_1min_1min_normalized,
    phase2_31_30s_30s_normalized,
    phase2_31_5min_5min_normalized,
    phase2_31_1min_5min_normalized
], ignore_index=True)

In [81]:
phase2_31_quantile1.to_csv('phase2_31_qn(5.18).csv',index=False)

In [75]:
phase2_31_quantile1.shape

(8426, 726)

In [42]:
## no 10min-10min plate 31v1
plates = ['31v1', '31v2','31v3']
data=phase2_df1[(phase2_df1['plate'].isin(plates))]
data.shape

(8809, 726)

In [77]:
phase2_31_quantile1[(phase2_31_quantile1['light_regime']=='2h-2h')&(phase2_31_quantile1['plate']=='33v2')]

,plate,measurement,start_date,light_regime,dark_threshold,light_threshold,num_frames,i,j,fv_fm,...,measurement_time_173,measurement_time_174,measurement_time_175,measurement_time_176,measurement_time_177,well_id,mutant_ID,feature,mutated_genes,num_mutations
2298,33v2,M4,2024-10-23,2h-2h,15.539387,22.378956,100,0,1,0.538348,...,NaN,NaN,NaN,NaN,NaN,A02,LMJ.RY0402.065793,intron,Cre13.g588600,1.0
2299,33v2,M4,2024-10-23,2h-2h,15.539387,22.378956,100,0,2,0.676217,...,NaN,NaN,NaN,NaN,NaN,A03,LMJ.RY0402.044988,CDS,Cre16.g664150,1.0
2300,33v2,M4,2024-10-23,2h-2h,15.539387,22.378956,100,0,3,0.686880,...,NaN,NaN,NaN,NaN,NaN,A04,LMJ.RY0402.168539,MULTIPLE_SPLICE_VARIANTS,Cre16.g693600,1.0
2301,33v2,M4,2024-10-23,2h-2h,15.539387,22.378956,100,0,4,0.624859,...,NaN,NaN,NaN,NaN,NaN,A05,LMJ.RY0402.098695,intron,Cre07.g334750,1.0
2302,33v2,M4,2024-10-23,2h-2h,15.539387,22.378956,100,0,5,0.655458,...,NaN,NaN,NaN,NaN,NaN,A06,LMJ.RY0402.238883,MULTIPLE_SPLICE_VARIANTS,Cre06.g256300,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2676,33v2,M4,2024-10-23,2h-2h,15.539387,22.378956,100,15,19,0.636822,...,NaN,NaN,NaN,NaN,NaN,P20,LMJ.RY0402.181834,intron,Cre02.g078939,1.0
2677,33v2,M4,2024-10-23,2h-2h,15.539387,22.378956,100,15,20,0.650225,...,NaN,NaN,NaN,NaN,NaN,P21,LMJ.RY0402.088794,intron,Cre12.g526850,1.0
2678,33v2,M4,2024-10-23,2h-2h,15.539387,22.378956,100,15,21,0.530679,...,NaN,NaN,NaN,NaN,NaN,P22,LMJ.RY0402.218008,CDS,Cre08.g374250,1.0
2679,33v2,M4,2024-10-23,2h-2h,15.539387,22.378956,100,15,22,0.688264,...,NaN,NaN,NaN,NaN,NaN,P23,LMJ.RY0402.118852,intron,"Cre08.g372100,Cre04.g214501",2.0
